In [1]:
%load_ext autotime
%matplotlib inline

In [4]:
import os
from pyspark.sql import SparkSession
import pymystem3
from pyspark.sql.types import BooleanType, IntegerType, StringType, FloatType, ArrayType
from pyspark.sql.functions import dayofyear, weekofyear, to_date, concat_ws, collect_list, year, udf, col, size, countDistinct, lit, sum, expr 
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer 
import re
from pyspark.ml.feature import NGram

import multiprocessing as mp
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.wordnet import WordNetLemmatizer

import artm
import glob
import os
import pandas as pd
from pyarrow import parquet

from pymystem3 import Mystem

time: 826 ms


In [5]:
full_datasets_path = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/'

time: 412 µs


/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
dataset_names = ["20newsgroups",
                 "amazon_food",
                 "hotel-reviews",
                 "lenta_ru",
                 "ads_data"
                ]

time: 396 µs


In [ ]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_NEW/20newsgroups

In [ ]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/test_set_data/

In [15]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

time: 1.54 s


# Processing instruments

In [8]:
# dataset cleaning 
r_num = re.compile(r'([0-9]+)')
r_punct = re.compile(r'[."\[\]/,()!?;:*#|\\%^$&{}~_`=-@]')
r_white_space = re.compile(r'\s{2,}')
r_words = re.compile(r'\W+')
re1 = re.compile(r'  +')
url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
r_rus = re.compile(r'[а-яА-Я]\w+')
r_html = re.compile(r'(\<[^>]*\>)')
r_email = re.compile(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)")

stop = stopwords.words("russian") + ['который', 'это', 'весьма', 'вполне', 'наверное']

m = Mystem()

def process_punkt(text):
    text = r_punct.sub(" ", text)
    text = r_vk_ids.sub(" ", text)
    text = r_num.sub(" ", text)
    text = r_white_space.sub(" ", text)
    return text.strip()


def new_html(text):
    text = r_html.sub("", text)
    return text


def lemmatize_text_ru(text):

    try:
        text = new_html(text)
    except:
        return ''

    text = text.lower()
    text = process_punkt(text)
    text = re.findall(r_rus, text)
    text = ' '.join(text)

    try:
        tokens = r_words.split(text)
    except:
        return ''

    tokens = (x for x in tokens if len(x) >= 2 and not x.isdigit())
    text = ' '.join(tokens)
    tokens = m.lemmatize(text)
    tokens = (x for x in tokens if x not in stop)
    tokens = (x for x in tokens if x.isalpha())
    text = ' '.join(tokens)

    return text

def lemmatize_text_en(line):
    try:
        line = new_html(line)
    except:
        return ''
    lmtzr = WordNetLemmatizer()    
    stop = stopwords.words('english')
    text_token = CountVectorizer().build_tokenizer()(line.lower())
    text_token = (x for x in text_token if len(x) > 2 and not x.isdigit())
    text_rmstop = (i for i in text_token if i not in stop)
    text_rmstop = (x for x in text_rmstop if x.isalpha())
    text_stem = ' '.join([lmtzr.lemmatize(w) for w in text_rmstop])
    #stext_stem = remove_more_html(text_stem)
    return text_stem
    

def text_to_tokens(text):
    return text.split()


def remove_more_html(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('<', ' ').replace('>', ' ').replace('#36;', '$').replace(
        '\\n', "\n").replace('quot;', "'").replace('<br />', "\n").replace(
        '\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ').replace('img', ' ').replace('class', ' ').replace(
        'src', ' ').replace('alt', ' ').replace('email', ' ').replace('icq', ' ').replace(
        'href', ' ').replace('mem', ' ').replace('link', ' ').replace('mention', ' ').replace(
        'onclick', ' ').replace('icq', ' ').replace('onmouseover', ' ').replace(
        'local', ' ').replace('key', ' ').replace('target', ' ').replace('amp', ' ').replace(
        'section', ' ').replace('search', ' ').replace('css', ' ').replace('style', ' ').replace(
        'cc', ' ').replace('text', ' ').replace("img", ' ').replace("expand", ' ').replace(
        "text", ' ').replace('\n', ' ').replace('dnum', ' ').replace('xnum', ' ').replace(
        'nnum', ' ').replace("ubuntu", ' ').replace('server', ' ').replace('port', ' ').replace('nntp', ' ')
    return re1.sub(' ', html.unescape(x))


def clear_url(text):
    return re.sub(url_pattern, ' ', text)


def tokens_count(text):
    return len(text.split())


def get_rid_of_num(text):
    tokens = text.split()
    tokens = [i for i in tokens if i != 'num']
    return ' '.join(tokens)

/opt/conda/lib/python3.7/site-packages/socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Callable
Installing mystem to /home/jovyan/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


time: 4.87 s


In [9]:
def clean_dataset(dataset_path, save_dataset_path, column_name, language):
    dataset = pd.read_csv(dataset_path)
    p = mp.Pool(mp.cpu_count())
    if language == "ru":
        dataset['processed_text'] = p.map(lemmatize_text_ru, dataset[column_name])
    if language == 'en':
        dataset['processed_text'] = p.map(lemmatize_text_en, dataset[column_name])
    dataset['tokens_count'] = p.map(tokens_count, dataset['processed_text'])
    
    print('Initial dataset shape: ', dataset.shape)
    dataset = dataset[dataset['tokens_count'] >= 3]
    print('Filtered dataset shape: ', dataset.shape)
    
    dataset.to_csv(save_dataset_path, index=None)
    print('Dataset is saved')
    
    return
    

def get_words_dict(text, stop_list):
    
    all_words = text
    words = sorted(set(all_words)-stop_list)
    
    return {w: all_words.count(w) for w in words}

def return_string_part(name_type, text):
    
    tokens = text.split()
    tokens = [item for item in tokens if item != '']
    tokens_dict = get_words_dict(tokens, set())
    
    return " |" + name_type + ' ' + ' '.join(['{}:{}'.format(k, v) for k, v in tokens_dict.items()])

def prepare_voc(batches_dir, vw_path, data_path, column_name='processed_text'):
    
    print('Starting...')
    with open(vw_path, 'w', encoding='utf8') as ofile:
        num_parts = 0
        try:
            for file in os.listdir(data_path):
                if file.startswith('part'):
                    print('part_{}'.format(num_parts), end='\r')
                    if file.split('.')[-1] == 'csv':
                        part=pd.read_csv(os.path.join(data_path, file))
                    else:
                        part = pd.read_parquet(os.path.join(data_path, file))
                    part_processed = part[column_name].tolist()
                    for text in part_processed:
                        result = return_string_part('@default_class', text)
                        ofile.write(result + '\n')
                    num_parts += 1

        except NotADirectoryError:
            print('part 1/1')
            part = pd.read_csv(data_path)
            part_processed = part[column_name].tolist()
            for text in part_processed:
                result = return_string_part('@default_class', text)
                ofile.write(result + '\n')

    print(' batches {} \n vocabulary {} \n are ready'.format(batches_dir, vw_path))
    
def prepare_batch_vectorizer(batches_dir, vw_path, data_path, column_name='processed_text'):
#     if not glob.glob(os.path.join(batches_dir, "*")):
    prepare_voc(batches_dir, vw_path, data_path, column_name=column_name)
    batch_vectorizer = artm.BatchVectorizer(data_path=vw_path,
        data_format="vowpal_wabbit", 
        target_folder=batches_dir,
    batch_size=100)
#     else:
#         batch_vectorizer = artm.BatchVectorizer(data_path=batches_dir, data_format='batches')
        
    return batch_vectorizer

def vocab_preparation(VOCAB_PATH, DICTIONARY_PATH):
    
    if not os.path.exists(VOCAB_PATH):
        with open(DICTIONARY_PATH, 'r') as dictionary_file:
            with open(VOCAB_PATH, 'w') as vocab_file:
                dictionary_file.readline()
                dictionary_file.readline()
                for line in dictionary_file:
                    elems = re.split(', ', line)
                    vocab_file.write(' '.join(elems[:2]) + '\n')

                    
def prepearing_cooc_dict(BATCHES_DIR, WV_PATH, VOCAB_PATH, COOC_DICTIONARY_PATH, 
                         cooc_file_path_tf, cooc_file_path_df, ppmi_dict_tf, ppmi_dict_df):
    print(f'BATCHES_DIR: {BATCHES_DIR}')
    print(f'WV_PATH: {WV_PATH}')
    print(f'VOCAB_PATH: {VOCAB_PATH}')
    print(f'COOC_DICTIONARY_PATH: {COOC_DICTIONARY_PATH}')
    print(f'cooc_file_path_tf: {cooc_file_path_tf}')
    print(f'cooc_file_path_df: {cooc_file_path_df}')
    print(f'ppmi_dict_tf: {ppmi_dict_tf}')
    print(f'ppmi_dict_df: {ppmi_dict_df}')
    
    ! bigartm -v $VOCAB_PATH -c $WV_PATH --cooc-window 10 --write-cooc-tf $cooc_file_path_tf --write-cooc-df $cooc_file_path_df --write-ppmi-tf $ppmi_dict_tf --write-ppmi-df $ppmi_dict_df
    
    cooc_dict = artm.Dictionary()
    cooc_dict.gather(
        data_path=BATCHES_DIR, 
        cooc_file_path=ppmi_dict_tf,
        vocab_file_path=VOCAB_PATH,
        symmetric_cooc_values=True)
    cooc_dict.save_text(COOC_DICTIONARY_PATH)
                    
        
def prepare_all(DATASET_PATH, column_name, dataset_lang):
    
    SAVE_DATASET_PATH = os.path.join(DATASET_PATH, 'processed_dataset.csv')
    BATCHES_DIR = os.path.join(DATASET_PATH, 'batches')
    WV_PATH = os.path.join(DATASET_PATH, 'wv.txt')
    DOCUMENTS_TO_BATCH_PATH = os.path.join(DATASET_PATH, 'dataset.csv')
    DICTIONARY_PATH = os.path.join(DATASET_PATH, 'dictionary.txt')
    VOCAB_PATH = os.path.join(DATASET_PATH, 'vocab.txt')
    COOC_DICTIONARY_PATH = os.path.join(DATASET_PATH, 'cooc_dictionary.txt')
    cooc_file_path_tf = os.path.join(DATASET_PATH, 'cooc_tf.txt')
    cooc_file_path_df = os.path.join(DATASET_PATH, 'cooc_df.txt')
    ppmi_dict_tf = os.path.join(DATASET_PATH,'ppmi_tf.txt')
    ppmi_dict_df = os.path.join(DATASET_PATH, 'ppmi_df.txt')
    
    clean_dataset(DOCUMENTS_TO_BATCH_PATH, SAVE_DATASET_PATH, column_name, dataset_lang)
    
    batch_vectorizer = prepare_batch_vectorizer(BATCHES_DIR, WV_PATH, DOCUMENTS_TO_BATCH_PATH)
    print('Batches are ready')
    
    my_dictionary = artm.Dictionary()
    my_dictionary.gather(data_path=BATCHES_DIR, vocab_file_path=WV_PATH)
    my_dictionary.filter(min_df=3, max_df=0.95, class_id='text')
    my_dictionary.save_text(DICTIONARY_PATH)
    print('Dictionary is ready')
    
    vocab_preparation(VOCAB_PATH, DICTIONARY_PATH)
    prepearing_cooc_dict(BATCHES_DIR, WV_PATH, VOCAB_PATH, 
                         COOC_DICTIONARY_PATH, cooc_file_path_tf, 
                         cooc_file_path_df, ppmi_dict_tf, 
                         ppmi_dict_df)

time: 3.06 ms


/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 20 Newsgroups full

In [10]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/amazon_food

Reviews.csv		      cooc_dictionary.txt   ppmi_df.txt
Reviews_10000_lemmatized.csv  cooc_tf.txt	    ppmi_tf.txt
batches			      dictionary.txt	    test_set_data_voc.txt
cooc_df.txt		      mutual_info_dict.pkl  vocab.txt
time: 480 ms


In [11]:
newsgroups = pd.read_csv('/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/20newsgroups/original_dataset/prepared.csv')

time: 430 ms


In [ ]:
! cp /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/20newsgroups/original_dataset/prepared.csv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/

In [ ]:
! mv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/prepared.csv /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/dataset.csv

In [ ]:
clean_dataset()

In [16]:
NG_DATASET = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/'

time: 318 µs


In [17]:
ng_column_name = 'processed_text'
dataset_lang = 'en'

time: 382 µs


In [18]:
prepare_all(NG_DATASET, ng_column_name, dataset_lang)

Initial dataset shape:  (11299, 5)
Filtered dataset shape:  (11280, 5)
Dataset is saved
Starting...
part 1/1
 batches /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/batches 
 vocabulary /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/wv.txt 
 are ready
Batches are ready
Dictionary is ready
BATCHES_DIR: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/batches
WV_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/wv.txt
VOCAB_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/vocab.txt
COOC_DICTIONARY_PATH: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/cooc_dictionary.txt
cooc_file_path_tf: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/cooc_tf.txt
cooc_file_path_df: /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/cooc_df.txt
p

In [ ]:
DATASET_PATH = NG_DATASET

SAVE_DATASET_PATH = os.path.join(DATASET_PATH, 'processed_dataset.csv')
BATCHES_DIR = os.path.join(DATASET_PATH, 'batches')
WV_PATH = os.path.join(DATASET_PATH, 'wv.txt')
DOCUMENTS_TO_BATCH_PATH = os.path.join(DATASET_PATH, 'dataset.csv')
DICTIONARY_PATH = os.path.join(DATASET_PATH, 'dictionary.txt')
VOCAB_PATH = os.path.join(DATASET_PATH, 'vocab.txt')
COOC_DICTIONARY_PATH = os.path.join(DATASET_PATH, 'cooc_dictionary.txt')
cooc_file_path_tf = os.path.join(DATASET_PATH, 'cooc_tf.txt')
cooc_file_path_df = os.path.join(DATASET_PATH, 'cooc_df.txt')
ppmi_dict_tf = os.path.join(DATASET_PATH,'ppmi_tf.txt')
ppmi_dict_df = os.path.join(DATASET_PATH, 'ppmi_df.txt')

In [20]:
! ls $NG_DATASET

batches		     cooc_tf.txt     ppmi_df.txt	    vocab.txt
cooc_df.txt	     dataset.csv     ppmi_tf.txt	    wv.txt
cooc_dictionary.txt  dictionary.txt  processed_dataset.csv
time: 407 ms


In [ ]:
! head -5 /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/20newsgroups_sample/test_set_data_voc.txt

# Testing

In [ ]:
TEST_FOLDER = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/test_folder'
TEST_DATASET_PATH = '/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/20newsgroups_sample'

In [ ]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/20newsgroups_sample

In [ ]:
TEST_WV_PATH = os.path.join(TEST_DATASET_PATH, 'test_set_data_voc.txt')
TEST_VOCAB_PATH = os.path.join(TEST_DATASET_PATH, 'vocab.txt')
test_cooc_file_path_tf = os.path.join(TEST_FOLDER, 'cooc_tf.txt')
test_cooc_file_path_df = os.path.join(TEST_FOLDER, 'cooc_df.txt')
test_ppmi_dict_tf = os.path.join(TEST_FOLDER, 'ppmi_tf.txt')
test_ppmi_dict_df = os.path.join(TEST_FOLDER, 'ppmi_df.txt')

In [ ]:
! head -5 $TEST_WV_PATH

In [ ]:
! bigartm -c $TEST_WV_PATH -v $TEST_VOCAB_PATH --cooc-window 10 --write-cooc-tf $test_cooc_file_path_tf --write-cooc-df $test_cooc_file_path_df --write-ppmi-tf $test_ppmi_dict_tf --write-ppmi-df $test_ppmi_dict_df 

In [ ]:
! ls $TEST_FOLDER

In [ ]:
! bigartm

In [ ]:
! head -5 /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/vocab.txt

In [ ]:
! head -5 /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups/wv.txt

In [ ]:
! rm -r bigartm.WARNING

In [ ]:
! ls /mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets_full/20newsgroups

In [ ]:
amazon_food = pd.read_csv('/mnt/ess_storage/DN_1/storage/home/khodorchenko/GOTM/datasets/amazon_food/Reviews.csv')

In [ ]:
amazon_food

In [ ]:
! ls /mnt/ess_storage/DN_1/archive/jupyter-notebooks-133/mailru/kh-mruml/notebooks/tm/Quality_Assessment

In [ ]:
! bigartm